In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import warnings 
warnings.filterwarnings(action = 'ignore') 

In [2]:
content = pd.read_csv('content.csv', error_bad_lines = False)
content.head()

,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad..."


In [4]:
import string
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')

# select features from original dataset
cotent_texts = content[['title', 'article_content', 'meta_title', 'meta_description', 'meta_keywords']]
content['merged'] = cotent_texts.astype(str).apply(' '.join, 1).str.lower()

# remove punctuations
content['merged'] = content['merged'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# remove stopwords
stop_words = stopwords.words('english')
content['merged_words'] = content['merged'].apply(lambda x: [word for word in x.split(' ') if not word in stop_words])

# content['merged_words'] = content['merged'].apply(lambda x: x.split(' '))
content.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soojinkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,entry_type,title,article_content,meta_title,meta_description,meta_keywords,merged,merged_words
0,/g26ftzdilh/bfunderstanding-dyslexia,Deep Dive Article,Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Understanding Dyslexia in Children | Dyslexia ...,Have you asked yourself what is dyslexia? Lear...,"dyslexia, what is dyslexia, signs of dyslexia,...",understanding dyslexia raising a child with dy...,"[understanding, dyslexia, raising, child, dysl..."
1,/g26ftzdilh/blazingly-fast-understanding-dyslexia,Deep Dive Article,Blazingly Fast Understanding Dyslexia,Raising a child with dyslexia can stir up a lo...,Entender la dislexia | ¿Qué es la discapacidad...,La dislexia es una condición real del cerebro ...,"Definición de la dislexia, qué es la dislexia,...",blazingly fast understanding dyslexia raising ...,"[blazingly, fast, understanding, dyslexia, rai..."
2,/learning-attention-issues/getting-started/wha...,Deep Dive Article,Awareness Month 2018: Supporting the 1 in 5,When it comes to learning and attention issues...,Awareness Month 2018: Supporting the 1 in 5,Understood is celebrating ADHD and learning di...,"learning disability awareness, adhd awareness,...",awareness month 2018 supporting the 1 in 5 whe...,"[awareness, month, 2018, supporting, 1, 5, com..."
3,/learning-attention-issues/getting-started/wha...,Deep Dive Article,"Helping Your Child, and You, #BeUnderstood","October is Learning Disabilities, ADHD, and Dy...","Helping Your Child, and You, #BeUnderstood",When kids with learning and attention issues a...,"BeUnderstood, #BeUnderstood, Be Understood, Un...",helping your child and you beunderstood octobe...,"[helping, child, beunderstood, october, learni..."
4,/learning-attention-issues/child-learning-disa...,Deep Dive Article,Understanding ADHD,What is ADHD? ADHD is a common condition that ...,What Is ADHD,"What is ADHD, and how does it affect kids? Use...","adhd, add, what is adhd, whats adhd, define ad...",understanding adhd what is adhd adhd is a comm...,"[understanding, adhd, adhd, adhd, common, cond..."


In [5]:
"""
size: The number of dimensions of the embeddings and the default is 100.
window: The maximum distance between a target word and words around the target word. The default window is 5.
min_count: The minimum count of words to consider when training the model; 
           words with occurrence less than this count will be ignored. 
           The default for min_count is 5.
workers: The number of partitions during training and the default workers is 3.
sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.
"""

word2vec = Word2Vec(content['merged_words'], min_count=3, size=32, workers=3, window=3, sg=0) # CBOW
print(word2vec.similarity('dyslexia', 'adhd'))
print(word2vec.similarity('dyslexia', 'understanding'))

0.5027946
0.33133355


In [ ]:
word_list = list(word2vec.wv.vocab.keys())
word_embeddings = pd.Series(word_list, index=word_list).apply(lambda x: word2vec[x])
df = pd.DataFrame.from_dict(word_embeddings.to_dict()).T
colnames = ['embed_' + str(i) for i in range(32)]
df.columns = colnames
df.to_csv('word2vec_stopwords_removed')